## Korfali et al. 2010
### Mouse leukocytes

In [4]:
import requests, json
import numpy as np
import pandas as pd
from time import sleep
from my_utils import get_url, find_duplicate
import my_config

In [12]:
# Excel import
df_Korfali_2010 = pd.read_excel('./SourceData/Papers/Korfali_2010/rk1-139_fromTableS5.xlsx')

# convert to df
df_Korfali_2010= pd.DataFrame(df_Korfali_2010.iloc[1:, 1])
df_Korfali_2010.columns = ['Gene_name']
df_Korfali_2010 = df_Korfali_2010.reset_index(drop=True)

In [13]:
# Manual inspection and correction
df_Korfali_2010.iloc[15, 0] = 'DHRS7'
df_Korfali_2010.iloc[39, 0] = 'NOC2L'
df_Korfali_2010.iloc[58, 0] = 'METTL7A'
df_Korfali_2010.iloc[104, 0] = 'TMEM189'

In [14]:
# find Uniprot Entry and gene names for each gene
for i in range(len(df_Korfali_2010)):
    # default values
    entry_converted = gene_obtained = 'Not_found'
    
    # gene name
    gene = df_Korfali_2010.iloc[i, 0]

    try:
        # Query the gene name to get the Uniprot ID and the Uniprot-registered gene name
        params = {
        "query": f'gene:{gene} AND organism_id:{my_config.organism_id_list["Homo sapiens"]}',
        "fields": "accession, gene_names",
        "format": "json"
        }
        
        r = get_url(my_config.WEBSITE_API, params=params)
        result = r.json()['results'][0]
        entry_converted = result['primaryAccession']
        gene_obtained = result['genes'][0]['geneName']['value']
    
    except Exception as e:
        print(f'Error in fetching {gene}: {e}') 
    
    # Put the obtained Entry and gene name
    df_Korfali_2010.loc[i, 'Entry_Korfali_2010'] = entry_converted
    df_Korfali_2010.loc[i, 'Gene_name_obtained'] = gene_obtained
    
    if i % 40 == 0: print(i, entry_converted, gene_obtained)
    
    # take a break and go next
    sleep(1)

0 Q9UH99 SUN2
40 Q3LXA3 TKFC
80 Q6NXT6 TAPT1
Error in fetching LOC100130633: list index out of range
Error in fetching TMPP: list index out of range
120 Q9Y2W6 TDRKH
Error in fetching MGC3196: list index out of range


### The three proteins that were not found seems to be removed from UniProt record. In the original paper they were hypothetical proteins
### I remove those three

In [15]:
df_Korfali_2010_cleaned = df_Korfali_2010[df_Korfali_2010.Entry_Korfali_2010 != 'Not_found']

In [23]:
df_Korfali_2010_cleaned = df_Korfali_2010_cleaned.drop(['Gene_name'], axis=1)

# Duplicate check
print(find_duplicate(df_Korfali_2010_cleaned.Entry_Korfali_2010.to_list()))

[]


In [19]:
df_Korfali_2010_cleaned.to_csv('./Output/Korfali_2010.csv', index=False)